### Creating a sqlite database, engine connection, and ORM classes

In [1]:
#dependencies 
import pandas as pd

#Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Column, Integer, String, Float, Date

In [2]:
#create engine 
engine = create_engine("sqlite:///hawaii.sqlite")

#create connection to the engine
conn = engine.connect()

In [3]:
#assign object to utilize declarative base
Base = declarative_base()

In [4]:
#create measurement and station classes
class Measurements(Base):
    __tablename__ = "measurements"
    
    id = Column(Integer, primary_key=True)
    station = Column(String(45))
    date = Column(String(45))
    prcp = Column(Float)
    tobs = Column(Integer)

    def __repr__(self):
        return f"id={self.id}, station={self.station}, prcp={self.prcp}"
    
class Stations(Base):
    __tablename__ = "stations"
    
    id = Column(Integer, primary_key=True)
    station = Column(String(45))
    name = Column(String(45))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, station={self.station}, name={self.name}"

In [5]:
#create metadata layer
Base.metadata.create_all(engine)

### Prepare CSV data

In [6]:
#read csv files
measurements_df = pd.read_csv("cleaned_hawaii_measurements.csv", dtype=object)
stations_df = pd.read_csv("cleaned_hawaii_stations.csv", dtype=object)

In [7]:
#convert dataframe files to list of dict
# orient="records" is used to create list of data
measurements_data = measurements_df.to_dict(orient="records")
stations_data = stations_df.to_dict(orient="records")

In [8]:
#check results of conversion for measurements
print(measurements_data[:5])

[{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': '0.08', 'tobs': '65'}, {'station': 'USC00519397', 'date': '2010-01-02', 'prcp': '0.0', 'tobs': '63'}, {'station': 'USC00519397', 'date': '2010-01-03', 'prcp': '0.0', 'tobs': '74'}, {'station': 'USC00519397', 'date': '2010-01-04', 'prcp': '0.0', 'tobs': '76'}, {'station': 'USC00519397', 'date': '2010-01-06', 'prcp': '0.0', 'tobs': '73'}]


In [9]:
#check results of conversion for stations
print(stations_data[:5])

[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': '21.2716', 'longitude': '-157.8168', 'elevation': '3.0'}, {'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': '21.4234', 'longitude': '-157.8015', 'elevation': '14.6'}, {'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': '21.5213', 'longitude': '-157.8374', 'elevation': '7.0'}, {'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': '21.3934', 'longitude': '-157.9751', 'elevation': '11.9'}, {'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': '21.4992', 'longitude': '-158.0111', 'elevation': '306.6'}]


### Load data into sqlite database

In [10]:
#use MetaData from SQLAlchemy to reflect tables 
metadata = MetaData(bind=engine)
metadata.reflect()

In [11]:
#create "measurement" and "station" dictionaries into two "table" variables
measurements_table = sqlalchemy.Table("measurements", metadata, autoload=True)
stations_table = sqlalchemy.Table("stations", metadata, autoload=True)

In [12]:
#remove pre-existing table
conn.execute(measurements_table.delete())
conn.execute(stations_table.delete())

In [13]:
#insert "measurements" data into the table to populate the table
conn.execute(measurements_table.insert(), measurements_data)

In [14]:
#check if data loaded into sqlite database by querying first five rows
conn.execute("Select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-06', 0.0, 73)]

In [15]:
#insert "stations" data into the table to populate the table
conn.execute(stations_table.insert(), stations_data)

In [16]:
conn.execute("Select * from stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]